In [50]:
import pandas as pd
import numpy as np


In [51]:
df = pd.read_csv("winequality-red.csv", sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [52]:
#--- 1. Data understanding

# Denna cell är för att ge oss ännu mer djupgående info så att vi kan undersöka datan och få en helhetsbild på det som ska undersökas utifrån statistikt tänk.
# Denna cell är även för att undersöka datan. Det man vill hitta är features och target. 
# Utifrån detta kan man se att vi har 11 features och 1 target vilket är ''quality''.

df.shape
df.info()
df.describe().round(2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00,1599.00
mean,8.32,0.53,0.27,2.54,0.09,15.87,46.47,1.00,3.31,0.66,10.42,5.64
std,1.74,0.18,0.19,1.41,0.05,10.46,32.90,0.00,0.15,0.17,1.07,0.81
min,4.60,0.12,0.00,0.90,0.01,1.00,6.00,0.99,2.74,0.33,8.40,3.00
25%,7.10,0.39,0.09,1.90,0.07,7.00,22.00,1.00,3.21,0.55,9.50,5.00
50%,7.90,0.52,0.26,2.20,0.08,14.00,38.00,1.00,3.31,0.62,10.20,6.00
75%,9.20,0.64,0.42,2.60,0.09,21.00,62.00,1.00,3.40,0.73,11.10,6.00
max,15.90,1.58,1.00,15.50,0.61,72.00,289.00,1.00,4.01,2.00,14.90,8.00


In [53]:
#Denna skapas för att visa klassfördelningar. Utifrån detta kan vi se en tydlig obalans pga de första två klasserna eftersom dessa har hundratals
#observationer medan andra bara har fåtal. Detta påverkar hur bra klassificieringsmodellen kan lära sig minoritetsklasserna.

#Varför är detta viktigt? Jo, pga accuracy blir missvisande (modellen kan få 50% accuracy genom klass 5 och 6) och k-NN påverkas mycket genom att minoritetspunkterna
#får vädigt få 'grannar' vilket leder till stor risk att bli felklassificerade.
df["quality"].value_counts()

quality
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

In [54]:
#--- 2. Data preparation

# I denna cell börjar vi gå till delen då vi ska börja förbereda datan för arbete. 
# Det vi är ute efter här: X= features och y=target. Vi ska skapa tränings- och testdata. normaliserad vs icke-normaliserad data.
# Vi behöver dela upp datan i X och y för att algoritmen ska lära sig sambandet.

X = df.drop ("quality", axis=1)

y = df["quality"]

In [55]:
# Vi måste även kontrollera att data + label är korrekt separerade. Detta gör vi med hjälp av denna cell.
X.shape, y.shape


((1599, 11), (1599,))

In [56]:
# Tanken är att vi ska tre olika splittar; 90%train-10%test | 66%train - 33%test | 50%train - 50%test. 
# Detta krävs för att jämföra hur datamängden påverkar kNN, visa att mindre testdata get instabilare accuracy och att visa att större träningsmängd
# ofta ger bättre resultat. 
# Det man har gjort här är att dela upp datan i träningsdelar och testdelar. Vi förbereder alltså datan för modellerna.
from sklearn.model_selection import train_test_split

X_train_90, X_test_10, y_train_90, y_test_10 = train_test_split(
    X, y, test_size=0.10, random_state=42
)

X_train_66, X_test_33, y_train_66, y_test_33 = train_test_split(
    X, y, test_size=0.33, random_state=42
)

X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(
    X, y, test_size=0.50, random_state=42
)

In [57]:
# Denna cell används endast som kontroll för att se hur många rader som hamnade i train och test.
len(X_train_90), len(X_test_10)

(1439, 160)

In [58]:
# Här importerar vi och skapar normaliseringsverktyget för att utföra normaliseringen i kommande steg.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [ ]:
# Det viktiga är att vi normaliserar endast features och inte target. En viktig detalj är att vi får inte normalisera hela datasetet för split för att detta
# skulle läcka information från testdatan till träningsdatan och detta får absolut inte hända.

# Vi normaliserar till intervallet [0,1] eftersom kNN och SVM använder avståndsberäkningar. 
# Om features har olika skalor kan avståndet snedvridas. Decision trees påverkas däremot knappt av normalisering vilket är bra.

# Denna normaliserar data för 90/10
scaler_90 = MinMaxScaler()
X_train_90_norm = scaler_90.fit_transform(X_train_90)
X_test_10_norm = scaler_90.transform(X_test_10)

# Denna normaliserar data för 66/33
scaler_66 = MinMaxScaler()
X_train_66_norm = scaler_66.fit_transform(X_train_66)
X_test_33_norm = scaler_66.transform(X_test_33)

# Denna normaliserar data för  50/50
scaler_50 = MinMaxScaler()
X_train_50_norm = scaler_50.fit_transform(X_train_50)
X_test_50_norm = scaler_50.transform(X_test_50)

# Denna kodsnutt testar normaliseringen och visar den nedan. dd
X_train_90_norm[:5]

array([[0.37168142, 0.08219178, 0.35      , 0.05479452, 0.07178631,
        0.16901408, 0.07420495, 0.28414097, 0.31496063, 0.15568862,
        0.44615385],
       [0.23893805, 0.1369863 , 0.23      , 0.09589041, 0.09015025,
        0.47887324, 0.22614841, 0.42657856, 0.54330709, 0.17365269,
        0.26153846],
       [0.46902655, 0.28082192, 0.57      , 0.10273973, 0.13522538,
        0.4084507 , 0.16254417, 0.51615272, 0.35433071, 0.25748503,
        0.49230769],
       [0.26548673, 0.3869863 , 0.23      , 0.09589041, 0.16527546,
        0.26760563, 0.27561837, 0.46475771, 0.37007874, 0.16766467,
        0.13846154],
       [0.38938053, 0.32876712, 0.29      , 0.07534247, 0.0951586 ,
        0.43661972, 0.23674912, 0.47503671, 0.47244094, 0.14371257,
        0.24615385]])

In [60]:
# Här importerar vi kNN + metrics för att utföra kommande delar.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


In [61]:
# Denna funktion är skapad just för att slippa skriva samma kod 18 gånger. Det blir mer strukturerat. 
# Sedan kan man loopa genom alla körningarna.
def run_knn(X_train, X_test, y_train, y_test, k):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    
    return cm, acc


In [62]:
# Det som ska ske här är att vi skapar 3 tester med olika k-värden som vi väljer. 2 dataset varianter alltså ickenormaliserad och normaliserad.
# Vi ska även ha med 3 train/test splittar. Totalt blir det alltså 18 körningar. Just därför la vi till funktionen ovan så att det ska bli smidigt att göra detta.
# Med varje körning ska vi även visa confusion matrix och accuracy.



In [63]:
# Vi börjar med 90/10 splitten med k = 3. Icke-normaliserad data.
cm_90_k3, acc_90_k3 = run_knn(X_train_90, X_test_10, y_train_90, y_test_10, k=3)

cm_90_k3, acc_90_k3


(array([[ 0,  0,  1,  0,  0,  0],
        [ 0,  1,  1,  1,  0,  0],
        [ 0,  2, 43, 21,  2,  0],
        [ 0,  4, 29, 28,  5,  0],
        [ 0,  0,  5,  8,  3,  2],
        [ 0,  0,  1,  1,  2,  0]], dtype=int64),
 0.46875)

In [64]:
# I denna del så ska vi loopa igenom alla 18 körningarna för att ha en smidig struktur. 
# Vi kommer att definiera alla tre k-värden, skapa en lista över alla train/test-splittar, skapa en lista över normaliseringstatus. Sedan kommer loopen att köras automatiskt.

results = []

k_values = [5, 17, 21]

splits = [
    ("90/10 original", X_train_90, X_test_10, y_train_90, y_test_10),
    ("90/10 norm", X_train_90_norm, X_test_10_norm, y_train_90, y_test_10),

    ("66/33 original", X_train_66, X_test_33, y_train_66, y_test_33),
    ("66/33 norm", X_train_66_norm, X_test_33_norm, y_train_66, y_test_33),

    ("50/50 original", X_train_50, X_test_50, y_train_50, y_test_50),
    ("50/50 norm", X_train_50_norm, X_test_50_norm, y_train_50, y_test_50),
]

for name, Xtr, Xte, ytr, yte in splits:
    for k in k_values:
        cm, acc = run_knn(Xtr, Xte, ytr, yte, k)
        results.append((name, k, acc, cm))



In [65]:
# Denna cell loopar alltså igenom resultatet och skriver ut en lista som presenterar körningen.
for r in results:
    print("Split:", r[0], "| k =", r[1])
    print("Accuracy:", round(r[2], 4))
    print("Confusion Matrix:")
    print(r[3])
    print("-" * 50)


Split: 90/10 original | k = 5
Accuracy: 0.475
Confusion Matrix:
[[ 0  0  1  0  0  0]
 [ 0  0  2  1  0  0]
 [ 0  1 41 23  3  0]
 [ 0  1 31 32  2  0]
 [ 0  0  5  9  3  1]
 [ 0  0  1  2  1  0]]
--------------------------------------------------
Split: 90/10 original | k = 17
Accuracy: 0.4813
Confusion Matrix:
[[ 0  0  1  0  0  0]
 [ 0  0  2  1  0  0]
 [ 0  0 42 24  2  0]
 [ 0  0 31 34  1  0]
 [ 0  0  6 11  1  0]
 [ 0  0  1  2  1  0]]
--------------------------------------------------
Split: 90/10 original | k = 21
Accuracy: 0.5375
Confusion Matrix:
[[ 0  0  1  0  0  0]
 [ 0  0  2  1  0  0]
 [ 0  0 46 20  2  0]
 [ 0  0 26 39  1  0]
 [ 0  0  6 11  1  0]
 [ 0  0  1  3  0  0]]
--------------------------------------------------
Split: 90/10 norm | k = 5
Accuracy: 0.5437
Confusion Matrix:
[[ 0  0  0  1  0  0]
 [ 0  0  1  2  0  0]
 [ 0  1 51 16  0  0]
 [ 0  1 31 28  6  0]
 [ 0  0  1  9  8  0]
 [ 0  0  0  2  2  0]]
--------------------------------------------------
Split: 90/10 norm | k = 17
Accu

In [66]:
# Efter att ha gjort alla 18 körningar sorterade vi resultatet efter accuracy. 
# Nu är målet att hitta de tre bästa körningarna (top 3 accuracy)
# Detta gör vi genom att: 1. Sortera resultaten, 2. plocka ut top 3, 3. Visa confusion matrix och accuracy för dem och 4. Förklara kort varför dessa presterade bäst

# Sortera resultaten efter accuracy (högst först)
sorted_results = sorted(results, key=lambda x: x[2], reverse=True)

# Plocka ut de tre bästa
top3 = sorted_results[:3]

# Denna del skriver ut dem snyggt
for r in top3:
    print("BÄSTA RESULTATET")
    print("Split:", r[0], "| k =", r[1])
    print("Accuracy:", round(r[2], 4))
    print("Confusion Matrix:")
    print(r[3])
    print("-" * 50)



# Det vi kan se här är att nästan alla toppresultat ligger på normaliserad data eftersom kNN beräknar avstånd. Dessutom påverkar val av k och mängden träningsdata resultatet mycket.

BÄSTA RESULTATET
Split: 90/10 norm | k = 17
Accuracy: 0.5813
Confusion Matrix:
[[ 0  0  1  0  0  0]
 [ 0  0  1  2  0  0]
 [ 0  0 50 18  0  0]
 [ 0  0 24 39  3  0]
 [ 0  0  2 12  4  0]
 [ 0  0  0  2  2  0]]
--------------------------------------------------
BÄSTA RESULTATET
Split: 90/10 norm | k = 21
Accuracy: 0.5563
Confusion Matrix:
[[ 0  0  1  0  0  0]
 [ 0  0  2  1  0  0]
 [ 0  0 48 20  0  0]
 [ 0  0 25 38  3  0]
 [ 0  0  0 15  3  0]
 [ 0  0  0  3  1  0]]
--------------------------------------------------
BÄSTA RESULTATET
Split: 50/50 norm | k = 5
Accuracy: 0.5563
Confusion Matrix:
[[  1   0   1   1   0   0]
 [  0   1  13  14   0   0]
 [  0   2 227 121   5   0]
 [  0   0  90 182  30   0]
 [  0   0  14  53  34   0]
 [  0   0   4   2   5   0]]
--------------------------------------------------


In [67]:
# Hur kan man läsa av detta?

# I vår bästa körning (90/10), normaliserad data, k 17, får vi en accuracy på ca 0.58.
# Om vi tittar på confusion matrixen ser vi att modellen framför allt lyckas med de vanligaste klasserna 5 och 6. 
# Till exempel klassificeras 50 av 68 femmor korrekt och 39 av 66 sexor korrekt.
# De ovanliga klasserna, som 3, 4 och 8, klassificeras däremot nästan alltid fel - ofta som 5 och 6.
# Detta hör självklart ihop med det som nämndes i början av labben alltså det är obalanserat vilket leder till att kNN tittar på närmaste grannar. 
# Eftersom det är 5 och 6 som dominerar, så kommer de andra siffrorna att blanda ihop sig med dem.
# När det bara finns några enstaka exempel av en klass kommer deras grannar ofta att tillhöra de deominerande klasserna.
# Slutsatsen är att confusion matrixen visar alltså att modellen fungerar hyfsat för de stora klasserna, men har dålig prestanda på minoritetsklasserna, trots att
# den här inställningen ger högst total accuracy.